# Reference implementation

This code defines a neural network for image classification, loads pre-trained weights and biases, processes a dataset of images, makes predictions, and writes the results to a CSV file.



In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


ModuleNotFoundError: No module named 'torch'

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
import csv

TENSOR_DIR = "../tensors"
WEIGHTS_AND_BIASES_DIR = "../weights_and_biases.txt"
PREDICTIONS_FILENAME = "predictions.csv"

class ExampleNN(nn.Module):
    """Fully connected neural network as described in README.md"""
    def __init__(self):
        super(ExampleNN, self).__init__()
        self.fc1 = nn.Linear(225, 98)
        self.fc2 = nn.Linear(98, 65)
        self.fc3 = nn.Linear(65, 50)
        self.fc4 = nn.Linear(50, 30)
        self.fc5 = nn.Linear(30, 25)
        self.fc6 = nn.Linear(25, 40)
        self.fc7 = nn.Linear(40, 52)

    def forward(self, x):
        # Cross-entropy loss only works with the logits directly, so we skip
        # the softmax for training
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = self.fc7(x)
        return x

    def forward_with_softmax(self, x):
        x = self.forward(x)
        return torch.softmax(x, dim=1)

class TensorDataset(Dataset):
    def __init__(self, tensor_dir, transform=None):
        self.tensor_dir = tensor_dir
        self.tensor_files = [f for f in os.listdir(tensor_dir)]
        self.transform = transform

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_name = self.tensor_files[idx]
        label = int(tensor_name[:2]) - 1

        tensor_path = os.path.join(self.tensor_dir, tensor_name)
        with open(tensor_path, 'r') as f:
            tensor_data = f.read().strip().split(',')
            tensor_data = [float(value) for value in tensor_data]
            image_tensor = torch.tensor(tensor_data, dtype=torch.float32)

        return image_tensor, label

def load_weights_and_biases(model, filename):
    """Load pre-provided weights and biases from a file"""
    state_dict = model.state_dict()
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in range(0, len(lines), 2):
            name = lines[i].strip().replace(':', '')
            values = list(map(float, lines[i+1].strip().split(',')))
            tensor_shape = state_dict[name].shape
            tensor = torch.tensor(values).view(tensor_shape)
            tensor = tensor.cuda()
            state_dict[name] = tensor
    model.load_state_dict(state_dict, strict=True)


dataset = TensorDataset(tensor_dir=TENSOR_DIR)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
model = ExampleNN().cuda()
load_weights_and_biases(model, WEIGHTS_AND_BIASES_DIR)

lookup_table = {
    0: 'A', 1: 'a', 2: 'B', 3: 'b', 4: 'C', 5: 'c', 6: 'D', 7: 'd',
    8: 'E', 9: 'e', 10: 'F', 11: 'f', 12: 'G', 13: 'g', 14: 'H', 15: 'h',
    16: 'I', 17: 'i', 18: 'J', 19: 'j', 20: 'K', 21: 'k', 22: 'L', 23: 'l',
    24: 'M', 25: 'm', 26: 'N', 27: 'n', 28: 'O', 29: 'o', 30: 'P', 31: 'p',
    32: 'Q', 33: 'q', 34: 'R', 35: 'r', 36: 'S', 37: 's', 38: 'T', 39: 't',
    40: 'U', 41: 'u', 42: 'V', 43: 'v', 44: 'W', 45: 'w', 46: 'X', 47: 'x',
    48: 'Y', 49: 'y', 50: 'Z', 51: 'z'
}

def predict(model, dataloader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model.forward_with_softmax(images)
            # select the most likely output from our softmax model
            _, preds = torch.max(outputs, 1)

            all_preds.append(
                (preds.item(), labels.item())
                )

    # order by the label
    sorted_preds = sorted(all_preds, key=lambda x: x[1])

    # map labels to letters using the lookup table
    mapped_preds = [(i + 1, lookup_table[guess]) for i, (guess, label) in enumerate(sorted_preds)]

    with open(PREDICTIONS_FILENAME, mode='w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(["image number", "label"])
      for label, guess in mapped_preds:
          print(f'{label=},  {guess=}')
          writer.writerow([label, guess])

predict(model, dataloader)

ModuleNotFoundError: No module named 'torch'